In [181]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn.preprocessing import StandardScaler
import random as ra
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tabulate import tabulate


# Load dataset

In [182]:
boston = load_boston()

data = pd.DataFrame(boston.data) 
data.columns = boston.feature_names 

In [183]:
data['Price'] = boston.target 


In [184]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [185]:
X = data.drop(['Price'], axis = 1)

Y = data[['Price']]


# Data Standardisation

In [186]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [187]:
Y= Y.values

# Train-Test Split

In [188]:
X_train,X_test,Y_train,Y_test = cross_validation.train_test_split(X,Y,test_size = 0.2)
Y_train = Y_train.reshape(Y_train.shape[0],)
Y_test = Y_test.reshape(Y_test.shape[0],)

In [189]:
print(Y_test.shape)
X_test.shape

(102,)


(102, 13)

# Initialising W from a normal distribution

In [190]:
Wo = np.random.normal(0, 1, X_train.shape[1])
Bo = ra.uniform(1,10)
Lr =0.001
#Wo = np.random.uniform(-1,1,X_train.shape[1])
print(Wo)

[-0.40303401  0.78394147  0.62801585  0.9283807   0.26390203  0.44945639
  1.54358151  1.54525183 -0.18790266  0.92023526  0.14682882  0.92873559
 -0.36988331]


# Gradient w.r.t W

In [191]:
def gradienCalc(X,Y,W,B,N,Lr):
    
    loss =  Y - (np.dot(X,W)+B)

    grad =  (2*(np.dot(loss,X))/N)*Lr

    return grad

# Gradient w.r.t B

In [192]:
def gradienCalcB(X,Y,W,B,N,Lr):
    
    loss =  Y - (np.dot(X,W)+B)

    gradB = ((2*np.sum(loss))/N)*Lr
    
    return gradB

# Mean Squared Loss

In [193]:
def lossCalc(X,Y,W,B,N):
    
    Loss = np.sum((Y - ((np.dot(X,W))+B))**2)
    
    return Loss/N

In [194]:
start = 0
batch = 30
iterations = 100000

test_Y = Y_train.reshape(-1,1)


# Computing SGD

In [195]:
for i in range(iterations):
    
 
    #=========================================================================================================================
    # If iterations increase, convergence is not happening because of overshooting, so after 5000 iterations learning rate is decreased to 000001
    
    if i > 5000:
        
        Lr = 0.000001;
    
    #=================================================================================
    
    # Batch SGD with batch size 10
    
    end = start+batch
    
    if (start >= X_train.shape[0]):
        
        start = 0;
        end   = batch;
        
    if (end > X_train.shape[0]):
        
        end = X_train.shape[0]
        

    mini_batch_X = X_train[start:end,:] 
        
    mini_batch_Y = test_Y[start:end,:] 

    mini_batch_Y = mini_batch_Y.reshape((end-start),)
        
    
  #================================================================================================      
    
    # Updating the Weights
    
    
    grad = gradienCalc(mini_batch_X,mini_batch_Y,Wo,Bo,mini_batch_X.shape[0],Lr)
    
    gradB = gradienCalcB(mini_batch_X,mini_batch_Y,Wo,Bo,mini_batch_X.shape[0],Lr)
  

    Wn = Wo + grad
    Bn = Bo + gradB
    
   #==============================================================================================

    dist = np.linalg.norm(Wo-Wn)

    if (dist < 0.00001 and abs(Bo-Bn)<0.00001):
        
        
        print("final Weight",Wo)
        
        print("Final iteration number ",i)
        
        print("Loss after final iteration on Train data ",lossCalc(X_train,Y_train,Wo,Bo,X_train.shape[0]))
        
        break;
        
    else:
        
        Wo = Wn
        Bo = Bn
        
        start = end+1

    if i>=99999:

        print("end of iterations")
   

final Weight [-6.21632482e-01  9.46435134e-01  1.08197484e-03  1.07841418e+00
 -1.49881989e+00  3.18357165e+00 -2.26026374e-01 -2.64878125e+00
  1.59525284e+00 -9.81386426e-01 -1.82403639e+00  8.28904004e-01
 -3.68945581e+00]
Final iteration number  5001
Loss after final iteration on Train data  23.032413752052666


In [196]:
print(Wo)
print(Bo)

[-6.21632482e-01  9.46435134e-01  1.08197484e-03  1.07841418e+00
 -1.49881989e+00  3.18357165e+00 -2.26026374e-01 -2.64878125e+00
  1.59525284e+00 -9.81386426e-01 -1.82403639e+00  8.28904004e-01
 -3.68945581e+00]
22.47164145487635


# Fitting Linear Regression For train data from sklearn

In [197]:
clf = LinearRegression(fit_intercept=True, normalize=False)
clf.fit(X_train, Y_train)
predicted = clf.predict(X_test)

# Comparing Mean squared error loss for SGD with SKLEARN on Test data

In [198]:
#========================================================

# Mean squared loss on Manual SGD model

finalLoss = lossCalc(X_test,Y_test,Wo,Bo,X_test.shape[0])

#=====================================================

# Mean squared loss on sklearn regression model

LrLoss = mean_squared_error(Y_test, predicted)

#=====================================================

print("Manual SGD model loss ", finalLoss)
print("sklearn regression loss", LrLoss)

Manual SGD model loss  20.16044300155029
sklearn regression loss 21.47884584994866


In [199]:
print(tabulate([["Final intercept",Bo,clf.intercept_ ], ["Final loss on test data", finalLoss,LrLoss]], headers=['Metric', 'SGD model','Sklearn model']))

Metric                     SGD model    Sklearn model
-----------------------  -----------  ---------------
Final intercept              22.4716          22.6679
Final loss on test data      20.1604          21.4788


In [200]:
print("Final weight after SGD ", Wo)
print("Final weight for sklearn regression  ", clf.coef_)

Final weight after SGD  [-6.21632482e-01  9.46435134e-01  1.08197484e-03  1.07841418e+00
 -1.49881989e+00  3.18357165e+00 -2.26026374e-01 -2.64878125e+00
  1.59525284e+00 -9.81386426e-01 -1.82403639e+00  8.28904004e-01
 -3.68945581e+00]
Final weight for sklearn regression   [-1.02532654  1.25215932  0.18352221  0.78413397 -2.44434557  2.54620177
  0.09540011 -3.65523423  2.7303766  -1.97145155 -2.15834563  0.8012498
 -4.00781444]
